In [1]:
# !pip install ./sacremoses/sacremoses-master/
# !pip install ./transformers/transformers-master/

STRIDE = 1
def is_jupyter():
    try:
        ipy_str = str(type(get_ipython()))
        if 'zmqshell' in ipy_str:
            return True
        
    except:
        return False

### Required Imports

I've added imports that will be used in training too

In [2]:
from sklearn.model_selection import train_test_split
import re
from random import shuffle as shfl
from auc import MyAUCCallback
from sklearn.utils import shuffle
from datetime import datetime
import pandas as pd
pd.set_option('display.max.columns', 500)
import numpy as np
import os
# os.environ['CUDA_VISIBLE_DEVICES'] ='3'
# os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-10.1/lib64'
import matplotlib.pyplot as plt
import gc
from shutil import copyfile
from catalyst.dl import SupervisedRunner, AlchemyLogger, CriterionCallback
from catalyst.dl.callbacks.metrics import AUCCallback
from torch.utils.data import DataLoader, SubsetRandomSampler,Dataset
batch_size =128
token = "d1dd16f08d518293bcbeddd313b49aa4"
DATA_DIR = '/kaggle/input/jigsaw-multilingual-toxic-comment-classification/'

/home/kb/jig_env/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



In [3]:
if os.uname()[1] == 'kb-Z370P-D3':
    # desktop
    LOG_PATH = '/media/ssd/logs/jigsaw'
    SERVER = False
    print('Working on desktop')
elif os.uname()[1] == 'kb-server':
    # server
    LOG_PATH = '/home/kb/logs/jigsaw'
    SERVER = True
    print('Working on server')
else:
    raise Exception('which hostname???')
    

Working on desktop


In [4]:
train = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
train = train[(train['toxic']>0.5) | (train['toxic']==0)]
train.toxic = train.toxic.round().astype(int)



In [5]:
df_valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')
    

In [6]:
# Combine train1 with a subset of train2
df_train = pd.concat([
    train[['comment_text', 'toxic']].query('toxic==1'),
    train[['comment_text', 'toxic']].query('toxic==0').sample(n=112226, random_state=0)
])
# df_train, df_valid_en = train_test_split(df_train, random_state=12345, 
#                                          stratify=df_train.toxic.values, test_size=0.2)

In [7]:
df_test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')\
    .rename(columns={'content':'comment_text'})

In [8]:
target_column = 'toxic'

### Define dataset

In [9]:
# 7from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import time
from torch.optim import lr_scheduler

import torch
from tqdm import tqdm
#import torch.utils.data as data
from torchvision import datasets, models, transforms
from transformers import *
import random
from math import floor, ceil
from sklearn.model_selection import GroupKFold

MAX_LEN = 96#192#192#512
SEP_TOKEN_ID = 102

class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, df, train_mode=True, labeled=True, train_transforms=None):
        
        self.train_transforms = train_transforms
        self.df = df
        if train_mode:
            self.labels = df.toxic.values
            self.toxic_inds = np.where(self.labels==1)[0]
            self.normal_inds = np.where(self.labels==0)[0]            
            
            
            print(f'Here is {len(self.labels)} samples, {len(self.toxic_inds)} samples and {len(self.normal_inds)} samples')
            print(f'Class balance is {len(self.toxic_inds)/len(self.labels):.2f}')
            
        self.train_mode = train_mode
        self.labeled = labeled

        self.tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')#, 


    def __getitem__(self, index):
        row = self.df.iloc[index]
        token_ids = self.get_token_ids(row)
        
        if self.labeled:
            labels = self.get_label(row)
            return {'features': token_ids, 'targets': labels}

        else:
            return {'features': token_ids}

    def __len__(self):
        return len(self.df)

    def trim_input(self, text, max_sequence_length=MAX_LEN):
        t = self.tokenizer.tokenize(text)
        t_len = len(t)

        if t_len + 2 > max_sequence_length:

            t_new_len = int(max_sequence_length) - 2

            t = t[:t_new_len]

        return t
        
    def get_token_ids(self, row):
        
        text = row.comment_text
        if self.train_transforms:
            lang = 'en' if 'lang' not in list(row.keys().values) else row.lang
            text, _ = self.train_transforms(data=(row.comment_text, lang))['data']
        
        if self.train_mode:
            token_ids = self.tokenizer.encode(text, max_length=1024)
            if len(token_ids) < MAX_LEN:
                ids = torch.tensor(token_ids + [0] * (MAX_LEN - len(token_ids)))
            else:
                ind_beg = random.randint(0, len(token_ids)-MAX_LEN)
                ids = torch.tensor(token_ids[ind_beg:ind_beg+MAX_LEN])
        else:
            token_ids = self.tokenizer.encode(text, max_length=MAX_LEN)
            if len(token_ids) < MAX_LEN:
                ids = torch.tensor(token_ids + [0] * (MAX_LEN - len(token_ids)))
            else:
                ids = torch.tensor(token_ids[:MAX_LEN])

        return ids

    def get_label(self, row):
        label = np.round(row[target_column])
        return torch.tensor([1-label, label]).float()
    
    def collate_fn(self, batch):
        token_ids = torch.stack([x[0] for x in batch])

        if self.labeled:
            labels = torch.stack([x[1] for x in batch])
            return {'features': token_ids, 'targets': labels}
        else:
            return {'features': token_ids}
        
 

## Build Model

In [10]:
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F


class QuestModel(nn.Module):
    def __init__(self, n_classes=2):
        super(QuestModel, self).__init__()
        self.model_name = 'QuestModel'
    
        self.bert_model = XLMRobertaModel.from_pretrained('xlm-roberta-large')

        self.fc = nn.Linear(1024, n_classes)

    def forward(self, ids):

        layers = self.bert_model(input_ids=ids)
        out = F.dropout(layers[0][:, 0, :], p=0.2, training=self.training)
        logit = self.fc(out)
        return logit
    


In [11]:
# bert_model = XLMRobertaModel.from_pretrained('xlm-roberta-base')

In [12]:
class BalancedSampler(torch.utils.data.sampler.Sampler):
    def __init__(self, dataset):
       
        self.toxic_inds = dataset.toxic_inds.copy()
        self.normal_inds = dataset.normal_inds.copy()
        
        self.num_samples = 2*max(len(self.toxic_inds), len(self.normal_inds))
        
        shfl(self.toxic_inds)
        shfl(self.normal_inds)
        
        self.inds = []
        for i in range(max(len(self.toxic_inds), len(self.normal_inds))):
            self.inds.append(self.normal_inds[i%len(self.normal_inds)])
            self.inds.append(self.toxic_inds[i%len(self.toxic_inds)])

    def __iter__(self):
        #print ('\tcalling Sampler:__iter__')
        return iter(self.inds)

    def __len__(self):
        #print ('\tcalling Sampler:__len__')
        return self.num_samples




In [13]:
def get_loaders(to_balance=True, shuffle_before=True):
    if SERVER:
        workers=1
    else:
        workers = 8    
    
    

    train_dataset = QuestDataset(df_train, train_mode=True)#, train_transforms=get_train_transforms())
    valid_dataset = QuestDataset(df_valid, train_mode=False)
    
       
    train_loader = DataLoader(
        train_dataset,
        num_workers=workers,
        sampler=None,
        batch_size=batch_size,
    )
    valid_loader = DataLoader(
        valid_dataset,
        num_workers=workers,
        batch_size=batch_size,
    )    
    
       
    loaders = {}
    loaders['train'] = train_loader
    loaders['valid'] = valid_loader
    
    
    for i in ['es', 'it', 'tr']:
        df = df_valid
        df = df[df['lang']==i]

        loaders['valid_'+ i] = DataLoader(
            QuestDataset(df, train_mode=False),
            num_workers=workers,
            batch_size=batch_size,
        )
    
    
    return loaders


In [14]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [16]:
def freeze_until(net, param_name):
    found_name = False
    for name, params in net.named_parameters():
        if name == param_name:
            found_name = True
        params.requires_grad = found_name
        
# param_name = 'bert_model.encoder.layer.10.attention.self.query.weight'
param_name = 'bert_model.encoder.layer.16.attention.self.query.weight'

# CHECKPOINT = './checkpoints/v93_xlm_roberta_large_ml96/best.pth'

project = "js_v13_engbias"
num_epochs = 300

group = datetime.now().strftime("%m%d%Y_%H%M%S")

    
if SERVER:
    group = f'{group}_srv'
    
if STRIDE > 1:
    group = f'{group}_str{STRIDE}'


gradient_accumulation_steps = 1

lr = 3e-5#1e-5#0.0001
group += f'_lr{lr}'

group = group.replace('.', '')

runner = SupervisedRunner(input_key=('features'), input_target_key=('targets'), output_key=('logits'))

experiment = 'simple'

torch.cuda.empty_cache()

logdir = f"{LOG_PATH}/{project}/{group}/{experiment}"

model = QuestModel(2)

# checkpoint = torch.load(CHECKPOINT)#, map_location=device)   
# model.load_state_dict(checkpoint['model_state_dict'])
# del checkpoint

freeze_until(model, param_name)

# model = model.to(device)


loaders = get_loaders(to_balance=True)


t_total = len(loaders['train'])//gradient_accumulation_steps*num_epochs
warmup_proportion = 0.01
num_warmup_steps = t_total * warmup_proportion

#     criterion = torch.nn.BCEWithLogitsLoss()
criterion = FocalLoss(alpha=0.2, gamma=1.5, logits=True, reduce=True)
#     criterion = torch.nn.CrossEntropyLoss()
#     criterion = torch.nn.BCELoss()
optimizer = AdamW(model.parameters(), lr = lr)
#    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=t_total) 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.25)
print(f'----------------Experiment: {experiment}')

runner.train(
    fp16=dict(opt_level="O1") ,
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
    verbose=True,
    distributed=False if is_jupyter() else True,
    callbacks=[
        AlchemyLogger(
                token=token, # your Alchemy token
                project=project,
                experiment=experiment,
                group=group,
            ),
        MyAUCCallback()

    ],
    main_metric='auc/_mean',
    minimize_metric=False,
    
    #fp16={"opt_level": "O1"}

)

Here is 224452 samples, 112226 samples and 112226 samples
Class balance is 0.50
----------------Experiment: simple
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
1/300 * Epoch (train):   0% 1/1754 [00:01<33:12,  1.14s/it, loss=0.027]

/home/kb/jig_env/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning:

size_average and reduce args will be deprecated, please use reduction='none' instead.

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning:

This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)



Early exiting                                                                
1/300 * Epoch (train):   7% 123/1754 [00:58<12:41,  2.14it/s, loss=7.211e-08]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/kb/jig_env/lib/python3.7/site-packages/catalyst/core/runner.py", line 497, in run_experiment
    self._run_stage(stage)
  File "/home/kb/jig_env/lib/python3.7/site-packages/catalyst/core/runner.py", line 474, in _run_stage
    self._run_epoch(stage=stage, epoch=state.epoch)
  File "/home/kb/jig_env/lib/python3.7/site-packages/catalyst/core/runner.py", line 451, in _run_epoch
    self._run_loader(loader)
  File "/home/kb/jig_env/lib/python3.7/site-packages/catalyst/core/runner.py", line 398, in _run_loader
    self._run_batch(batch)
  File "/home/kb/jig_env/lib/python3.7/site-packages/catalyst/core/runner.py", line 376, in _run_batch
    self._run_event("on_batch_end")
  File "/home/kb/jig_env/lib/python3.7/site-packages/catalyst/core/runner.py", line 329, in _run_event
    getattr(callback, event)(self.state)
  File "/home/kb/Documents/jigsaw/auc.py", line 82, in on_batch_end
    self.meters[0].add(probabilities[:, 1], targets[:, 1])
  F

TypeError: can only concatenate str (not "list") to str

In [ ]:
# for name, params in model.named_parameters():
#     print(name)

#### 

In [ ]:
# for i, row in df_train.iterrows():
#     print(list(row.keys().values))
#     break

In [ ]:
--

In [ ]:
CHECKPOINT = '/media/ssd/logs/jigsaw/js_v935_xlmrbtlg_10fr_pseudo/05_04_2020__14_16_58_lr1e-06_pseudo_train/simple/checkpoints/best.pth'

In [ ]:
model = QuestModel(2)

checkpoint = torch.load(CHECKPOINT)#, map_location=device)   

model.load_state_dict(checkpoint['model_state_dict'])
del checkpoint

In [ ]:
model = model.to(device)
_ = model.eval()

In [ ]:
df_test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')\
    .rename(columns={'content':'comment_text'})
ds = QuestDataset(df_test, train_mode=False, labeled=False)
loader = DataLoader(
        ds,
        num_workers=8,
        batch_size=batch_size,
    )    

In [ ]:
df_test.head()

In [ ]:
y_pred = torch.zeros( len(ds), 2).to(device)

j = 0
with torch.no_grad():
    for data in tqdm(loader):
        x = data['features'].to(device)
        thish_batch_size = len(x)            
        
        
        y_pred[j:j+thish_batch_size] = model(x)
            
        j += thish_batch_size

In [ ]:
from torch import sigmoid
y_pred_sigmoid = sigmoid(y_pred)

In [ ]:
y_pred_sigmoid_one_np = y_pred_sigmoid.cpu().numpy()

In [ ]:
df_sub['toxic'] = y_pred_sigmoid_one_np[:,1]

In [ ]:
df_sub.to_csv('submission.csv', index=False)

In [ ]:
df_sub